In [15]:
from multiprocessing import Pool
import time
import math

N = 5000000

def cube(x):
    return math.sqrt(x)

if __name__ == "__main__":
    # first way, using multiprocessing
    start_time = time.perf_counter()
    with Pool() as pool:
      result = pool.map(cube, range(10,N))
    finish_time = time.perf_counter()
    print("Program finished in {} seconds - using multiprocessing".format(finish_time-start_time))
    print("---")
    # second way, serial computation
    start_time = time.perf_counter()
    result = []
    for x in range(10,N):
      result.append(cube(x))
    finish_time = time.perf_counter()
    print("Program finished in {} seconds".format(finish_time-start_time))

Program finished in 1.1589245349168777 seconds - using multiprocessing
---
Program finished in 1.2017867527902126 seconds


# Parallel Sampling with Dynesty

In [8]:
import sys
sys.path.append('/mnt/pfs/akash.mishra/pytimedomain/')
import matplotlib.pyplot as plt
import readligo as rl
import matplotlib.mlab as mlab, matplotlib.pyplot as plt, numpy as np, os, sys, warnings

import numpy as np
from scipy.linalg import toeplitz
from Data import DataProcessor
import noise
from scipy.signal      import butter, filtfilt, welch, tukey, decimate
import likelihood_class

from noise import detector
import waveform
import lal
import matplotlib.pyplot as plt
import readligo as rl
import matplotlib.mlab as mlab, matplotlib.pyplot as plt, numpy as np, os, sys, warnings

import numpy as np
import pandas as pd
from scipy.linalg import toeplitz
from scipy.interpolate import interp2d

warnings.filterwarnings("ignore", category=DeprecationWarning)
import numpy as np
import dynesty
from multiprocessing import Pool
import numba

In [4]:
kwargs = {'f_min':20, 'f_max':2047, 'data_duration':4096,'triggertime':1126259462.423, 'srate':4096, 'signal_chunk_size':4, 'noise_chunk_size':4,
                'output': '/mnt/pfs/akash.mishra/pytimedomain/', 'window': 1, 'alpha-window': 0.1, 'fft-acf': True, 'acf-file':'', 'acf-simple-norm': False}

file_h1 = '/mnt/pfs/akash.mishra/pytimedomain/data_files/GW150914/H-H1_LOSC_4_V2-1126257414-4096.hdf5'


n_duration = 820
h1_data = detector('H1', file_h1, **kwargs)

file_l1 = '/mnt/pfs/akash.mishra/pytimedomain/data_files/GW150914/L-L1_LOSC_4_V2-1126257414-4096.hdf5'

l1_data = detector('L1', file_l1, **kwargs)
# cov_inverse = {'H1': np.linalg.inv(toeplitz(h1_data.acf[0:820])), 'L1': np.linalg.inv(toeplitz(l1_data.acf[0:820]))}


cov_inverse = {'H1': np.linalg.inv(toeplitz(h1_data.acf[0:n_duration])), 'L1': np.linalg.inv(toeplitz(l1_data.acf[0:n_duration]))}



def time_d(det_list, ref_det, ra, dec, tM_gps):
    td = {}
    ref_det_lal =  lal.cached_detector_by_prefix[ref_det]
    for d in det_list:
        det = lal.cached_detector_by_prefix[d]
        td[ref_det + '_' + d] = lal.ArrivalTimeDiff(det.location, ref_det_lal.location, ra, dec, tM_gps)
    return td


det_class = {'H1': h1_data, 'L1':l1_data}
# det_class = {'H1': h1_data, 'L1':l1_data}


ra = 1.95
dec = -1.27
psi = 0.82
trigger_time =  1126259462.423
t_start = 0.000

ref_det = 'H1' 
# time_delay = time_d(['H1', 'L1', 'V1'], ref_det, ra, dec,  trigger_time)
time_delay = time_d(['H1', 'L1'], ref_det, ra, dec,  trigger_time)

* Computing the one-sided PSD with the Welch method for comparison with the standard ACF.

* Computing the one-sided PSD with the Welch method for comparison with the standard ACF.



In [5]:
def interpolated_functionb(datafile):
    qnm_data = pd.read_pickle(datafile)
    q_list = list(qnm_data.keys())
    a_list = list(qnm_data[0.0].keys())
    x = a_list
    y = q_list  
    z_real = []
    z_imag = []
    for i in range(len(q_list)):
        z_real.append(tuple(np.real(list(qnm_data[q_list[i]].values())))[0:194])
        z_imag.append(tuple(np.imag(list(qnm_data[q_list[i]].values())))[0:194])
    
    wreal = interp2d(x, y, np.array(z_real, dtype=object))
    wimag = interp2d(x, y, np.array(z_imag, dtype=object))
    
    return wreal, wimag

In [6]:
real_220_interp, imag_220_interp = interpolated_functionb('/mnt/pfs/akash.mishra/pytimedomain/data_files/braneworld_qnm_files/qnm_data_220_updated.pickle')
real_221_interp, imag_221_interp = interpolated_functionb('/mnt/pfs/akash.mishra/pytimedomain/data_files/braneworld_qnm_files/qnm_data_221_updated.pickle')

In [7]:
import waveform
interpolants_bw = {(2,2,2,0):{'freq': real_220_interp, 'tau': imag_220_interp}, (2,2,2,1):{'freq': real_221_interp, 'tau': imag_221_interp}}
waveform.QNM_braneworld(2,2,2,0, interpolants_bw).f(70, 0.67, 0.0)

array([239.9691693])

In [12]:
# @numba.jit
def loglikelihood(theta):
    mass_f, chi, q,  amp0, amp1, iota, phi = theta
    amps = {(2,2,2,0): amp0, (2,2,2,1): amp1}
    phase = {(2,2,2,0): 0.0, (2,2,2,1): 0.0}
    r = 400
    
    logL = 0.0

    for d in det_class.keys():
        lal_det = lal.cached_detector_by_prefix[d]
        dt   = time_delay['{0}_'.format(ref_det)+d]
        tref = lal.LIGOTimeGPS(t_start+dt+trigger_time)
        time_array_raw = det_class[d].time - (trigger_time+dt)
        time_array     = time_array_raw[time_array_raw >= t_start][:n_duration]
        data           = det_class[d].time_series[time_array_raw >= t_start][:n_duration]
        hp, hc = waveform.BraneworldBH(t_start, mass_f, chi, q, amps, phase, r, iota, phi, interpolants_bw).waveform(time_array)
        prediction = likelihood_class.project(hp, hc, lal_det, ra, dec, psi, tref)
        dd = likelihood_class.inner_product_direct_inversion(data,       data,       cov_inverse[d])
        dh = likelihood_class.inner_product_direct_inversion(data,       prediction, cov_inverse[d])
        hh = likelihood_class.inner_product_direct_inversion(prediction, prediction, cov_inverse[d])
        residuals_inner_product = dd - 2. * dh + hh
        logL += -0.5*residuals_inner_product #+ log_normalisation
    return logL

# @numba.jit
def prior_transform(utheta):
    #mass_fm, chi_m, amp0_m, amp1_m, phi0_m, phi1_m, ra_m, dec_m, psi_m = utheta
    mass_fm, chi_m,  q_m, amp0_m, amp1_m, iota_m, phi_m = utheta
    mass_f = 20 + (150-20) * mass_fm
    q_min = -0.95
    q_max = 0.0
    q = q_min + (q_max - q_min) * q_m
    # a_min = 0.01
    # a_max = 0.98
    
    def a_max_func(x):
        return np.sqrt(1 - x)-0.02
    
    a_min = 0.05
    
    #a_max_func = lambda q: (2*(0.5*np.sqrt(1+4*(q_m)))-0.02)
    a_max = a_max_func(q)
    chi = a_min + (a_max - a_min) * chi_m
    
    
    #chi =  0.1 + (0.98 - 0.1) * chi_m
    amp0 = 1e-23 + (1e-19 - 1e-23) * amp0_m
    amp1 = 1e-23 + (1e-19 - 1e-23) * amp1_m
    # phi0 = 2 * np.pi * phi0_m
    # phi1 = 2 * np.pi * phi1_m

    iota =  np.pi * iota_m
    phi = 2 * np.pi * phi_m
    #ra = 2 * np.pi * ra_m
    #dec =  np.pi * (dec_m) - np.pi/2 #np.pi * dec_m
    #psi =  np.pi * psi_m
    
    return mass_f, chi, q, amp0, amp1, iota, phi

In [13]:
import dynesty.pool as dypool


In [16]:
ndim = 7

with dypool.Pool(4, loglikelihood, prior_transform) as pool:
    # The important thing that we provide the loglikelihood/prior transform from 
    # the pool    
    psampler = dynesty.DynamicNestedSampler(pool.loglike, pool.prior_transform, ndim, 
                                 nlive=1000, sample='rslice',pool=pool,
                                     )
    psampler.run_nested()
pres = psampler.results

10898it [21:12,  8.57it/s, batch: 0 | bound: 70 | nc: 32 | ncall: 297557 | eff(%):  3.656 | loglstar:   -inf < -827.102 <    inf | logz: -845.663 +/-  0.249 | dlogz:  0.084 >  0.010]  


KeyboardInterrupt: 